In [ ]:
#! pip install t5
#import requests
#
#subject = input("What is your topic?")

#url = 'https://en.wikipedia.org/w/api.php'
#params={
#    'action':'query',
#    'format':'json',
#    'titles': subject,
#    'prop':'extracts',
#    'exintro':True,
#    'explaintext': True,
#}
#response = requests.get(url, params=params)
#data = response.json()
#
#page = next(iter(data['query']['pages'].values()))
#
#print(page['extract'][:])




In [ ]:
import pandas as pd
import requests


df=pd.read_csv('../csv/output.csv',usecols='category')

for x in df:
    subject = df[1]


